# Running Federated Computations in the Cloud

Let's explore how easy it will be to modify Part A to support predictions in the cloud. We'll point out the differences below.

The first steps are to set up the cloud instances. There are few prerequisites that need to be setup prior to launching the cloud instances. This tutorial uses Google Cloud so you'll need an account set up there and need to have the gcloud command-line tool installed.

See [here](../private-prediction/CLOUD.md) for more details about installing the gcloud command-line tool and launching the instances.

NOTE: If unable to use Google Cloud, you can set the environment variables `INSTANCE_NAMES` and use the script `launch_local_servers.sh`

```
export INSTANCE_NAMES="server0 server1 crypto-producer model-owner data-owner-0 data-owner-1 data-owner-2"
./launch_local_servers.sh $INSTANCE_NAMES
```

In [ ]:
import tensorflow as tf
import tf_encrypted as tfe

config = tfe.RemoteConfig.load("/tmp/config.json")
config.connect_to_cluster()

tfe.set_config(config)
tfe.set_protocol(tfe.protocol.Pond())

from players import BaseModelOwner, BaseDataOwner
from func_lib import default_model_fn, secure_mean, evaluate_classifier
from util import split_dataset
from download import download_mnist

NUM_DATA_OWNERS = 3
BATCH_SIZE = 256
DATA_ITEMS = 60000
BATCHES = DATA_ITEMS // NUM_DATA_OWNERS // BATCH_SIZE
LEARNING_RATE = 0.01

In [ ]:
class ModelOwner(BaseModelOwner):
  @classmethod
  def model_fn(cls, data_owner):
    return default_model_fn(data_owner)

  @classmethod
  def aggregator_fn(cls, model_gradients, model):
    return secure_mean(model_gradients)

  @classmethod
  def evaluator_fn(cls, model_owner):
    return evaluate_classifier(model_owner)

class DataOwner(BaseDataOwner):
  pass

In [ ]:
download_mnist()
split_dataset("./data", NUM_DATA_OWNERS, DATA_ITEMS)

In [ ]:
model = tf.keras.Sequential((
    tf.keras.layers.Dense(512, input_shape=[None, 28 * 28],
                          activation='relu'),
    tf.keras.layers.Dense(10),
))

model.build()

loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(LEARNING_RATE)

In [ ]:
model_owner = ModelOwner("model-owner",
                         "{}/train.tfrecord".format("./data"),
                         model, loss,
                         optimizer=opt)

# Simplify this with a loop?
data_owners = [DataOwner("data-owner-{}".format(i),
                         "{}/train{}.tfrecord".format("./data", i),
                         model, loss,
                         optimizer=opt)
              for i in range(NUM_DATA_OWNERS)]

In [ ]:
model_owner.fit(data_owners, rounds=BATCHES, evaluate_every=10)

print("\nDone training!!")